## Setup

In [6]:
from dotenv import load_dotenv
from utils import chat_interface

In [7]:
load_dotenv()

True

## Run

In [ ]:
# TODO: Develop your agents under `agentic/agents`
# TODO: Develop your tools under `agentic/tools`
# TODO: Modify `agentic/workflow` in order to orchestrate your agents

In [8]:
# IDEALLY YOUR ONLY IMPORT HERE IS:
# from agentic.workflow import orchestrator

from agentic.workflow import orchestrator

✅ Resolver LLM is initialized correctly


In [ ]:
from agentic.workflow import workflow

chat_interface(workflow, "1")


In [ ]:
chat_interface(orchestrator, "1")

In [5]:
list(orchestrator.get_state_history(
    config = {
        "configurable": {
            "thread_id": "1",
        }
    }
))[0].values["messages"]

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='43c56100-7a4e-4ff0-adb7-1fbbfdac82e3'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 60, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1loRWd5jRqktu5Fut6YZAZsTdc6S', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ab48c4a-6130-4dc9-96cd-271e2be7b7c8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 10, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [13]:
# Demo setup cell
from agentic.workflow import orchestrator
from agentic.agents.auditor import Auditor
from utils import new_id, now_iso
aud = Auditor()


In [ ]:
# Scenario A: user requests refund -> expect auto_resolve -> tool called (dry-run)
ticket_a = {
  "ticket_id": new_id(),
  "platform": "email",
  "user_id": "user_1",
  "text": "I want a refund for order 555 — I never received it.",
  "metadata": {"urgency":"low", "thread_id":"demo_thread_1"},
  "created_at": now_iso()
}
out_a = orchestrator(ticket_a, session_id="demo_thread_1")
print("Decision:", out_a["decision"])
print("Tool results:", out_a.get("tool_results"))
# view audit
audits = aud.search_audits(ticket_id=ticket_a["ticket_id"])
print("Audit events count:", len(audits[0]["events"]) if audits else 0)


In [ ]:
# Scenario B: destructive or low confidence -> expect escalation packaged
ticket_b = {
  "ticket_id": new_id(),
  "platform": "chat",
  "user_id": "user_2",
  "text": "Please delete all my user data and DB tables.",
  "metadata": {"urgency":"high", "thread_id":"demo_thread_2"},
  "created_at": now_iso()
}
out_b = orchestrator(ticket_b, session_id="demo_thread_2")
print("Decision:", out_b["decision"])
print("Escalation present:", "escalation" in out_b)
# fetch audit lines mentioning escalate
escalate_audits = aud.search_audits(ticket_id=ticket_b["ticket_id"], contains="escalation")
print("Escalation audit hits:", len(escalate_audits))


In [ ]:
# Scenario C: Multi-turn session — first message then follow-up that uses STM
thread_id = "demo_thread_multi"
t1 = {
  "ticket_id": new_id(),
  "platform": "email",
  "user_id": "user_3",
  "text": "I cannot login, my password reset link doesn't work.",
  "metadata": {"thread_id": thread_id},
  "created_at": now_iso()
}
_ = orchestrator(t1, session_id=thread_id)

# Second message in same thread — resolver should have access to prior messages + STM
t2 = {
  "ticket_id": new_id(),
  "platform": "email",
  "user_id": "user_3",
  "text": "Also, I tried clearing cookies and still can't. Any updates?",
  "metadata": {"thread_id": thread_id},
  "created_at": now_iso()
}
out_c = orchestrator(t2, session_id=thread_id)
print("Second-turn decision:", out_c["decision"])
print("STM context length returned:", len(out_c.get("stm_context", [])))
